In [1]:
import requests
import bs4
import re
import pyarrow.parquet as pq
import pandas as pd
import geopandas as gpd
from skimpy import clean_columns

In [2]:
!pip install requests
!pip install pyarrow
!pip install pandas
!pip install geopandas
!pip install skimpy

You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


### Downloading the data

In [3]:
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

In [4]:
def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html


def find_taxi_parquet_links():
    soup = bs4.BeautifulSoup(get_taxi_html(), 'html.parser')
    result = [a['href'] for a in soup.find_all('a', title="Yellow Taxi Trip Records")]
    return result

In [6]:
yellow_taxi_url = find_taxi_parquet_links()
yellow_taxi_url

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-07.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-08.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-04.pa

In [7]:
result = []
for data in yellow_taxi_url:
    match = re.search(r"\w+-\w+\.parquet", data)
    result.append(match.group())

result

['yellow_tripdata_2022-01.parquet',
 'yellow_tripdata_2022-02.parquet',
 'yellow_tripdata_2022-03.parquet',
 'yellow_tripdata_2022-04.parquet',
 'yellow_tripdata_2022-05.parquet',
 'yellow_tripdata_2022-06.parquet',
 'yellow_tripdata_2022-07.parquet',
 'yellow_tripdata_2022-08.parquet',
 'yellow_tripdata_2021-01.parquet',
 'yellow_tripdata_2021-02.parquet',
 'yellow_tripdata_2021-03.parquet',
 'yellow_tripdata_2021-04.parquet',
 'yellow_tripdata_2021-05.parquet',
 'yellow_tripdata_2021-06.parquet',
 'yellow_tripdata_2021-07.parquet',
 'yellow_tripdata_2021-08.parquet',
 'yellow_tripdata_2021-09.parquet',
 'yellow_tripdata_2021-10.parquet',
 'yellow_tripdata_2021-11.parquet',
 'yellow_tripdata_2021-12.parquet',
 'yellow_tripdata_2020-01.parquet',
 'yellow_tripdata_2020-02.parquet',
 'yellow_tripdata_2020-03.parquet',
 'yellow_tripdata_2020-04.parquet',
 'yellow_tripdata_2020-05.parquet',
 'yellow_tripdata_2020-06.parquet',
 'yellow_tripdata_2020-07.parquet',
 'yellow_tripdata_2020-08.pa

In [8]:
### downloading the datasets programmatically 
"""
for data in yellow_taxi_url:
    response = requests.get(data, stream=True)
    file_name = re.search(r"\w+-\w+\.parquet", data)
    
    with open(file_name.group(), "wb") as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)
"""


'\nfor data in yellow_taxi_url:\n    response = requests.get(data, stream=True)\n    file_name = re.search(r"\\w+-\\w+\\.parquet", data)\n    \n    with open(file_name.group(), "wb") as f:\n        for chunk in response.iter_content(chunk_size=1024): \n            if chunk:\n                f.write(chunk)\n'

In [9]:
df_test_01_2022 = pd.read_parquet("yellow_tripdata_2022-01.parquet")
df_test_01_2022.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


### Cleaning and Filtering

##### 1. Looking up the latitude and longitude for some months where only location IDs are given for pickups and dropoffs (use the shp file linked above) – some location IDs may not be valid so those specific trips can be removed

In [9]:
# still waiting for the TAs to get back with the other .shp files

##### 2. Removing unnecessary columns and only keeping columns needed to answer questions in the other parts of this project

In [10]:
#will be done later on

##### 3. Removing invalid data points (use your discretion!)

In [11]:
#Check data points for each column

In [10]:
df_test_01_2022["VendorID"].value_counts() 
#Based on the data dictionary, the possible values for VendorID should only be 1 and 2
#We'll drop the 6 and 5 columns

2    1716059
1     742273
6       5563
5         36
Name: VendorID, dtype: int64

In [11]:
VendorID = list(range(1,3))
df_test_01_2022 = df_test_01_2022[df_test_01_2022.VendorID.isin(VendorID)]
df_test_01_2022.VendorID.value_counts()

2    1716059
1     742273
Name: VendorID, dtype: int64

In [12]:
df_test_01_2022["tpep_pickup_datetime"].isna().sum()
#all good for tpep_pickup_datetime

0

In [13]:
df_test_01_2022["tpep_dropoff_datetime"].isna().sum()
#all good for tpep_dropoff_datetime

0

In [14]:
df_test_01_2022["passenger_count"].value_counts()

1.0    1794055
2.0     343026
3.0      84570
0.0      52061
5.0      51338
4.0      35321
6.0      32037
7.0          9
8.0          8
9.0          3
Name: passenger_count, dtype: int64

In [15]:
df_test_01_2022["passenger_count"].isna().sum()

65904

In [16]:
#drop null value and 0.0 value from the respective column
#justification: It's impossible to have 0.0, otherwise there should be another service offered (e.g. send goods)
passenger_count = list(range(1,10))
df_test_01_2022 = df_test_01_2022[df_test_01_2022["passenger_count"].notna()] #drop null values
df_test_01_2022 = df_test_01_2022[df_test_01_2022["passenger_count"].isin(passenger_count)] #drop values of 0.0
df_test_01_2022["passenger_count"].value_counts()

1.0    1794055
2.0     343026
3.0      84570
5.0      51338
4.0      35321
6.0      32037
7.0          9
8.0          8
9.0          3
Name: passenger_count, dtype: int64

In [17]:
df_test_01_2022["trip_distance"].isna().sum()

0

In [18]:
df_test_01_2022["PULocationID"].isna().sum()

0

In [19]:
df_test_01_2022["DOLocationID"].isna().sum()

0

In [20]:
df_test_01_2022["RatecodeID"].isna().sum()

0

In [21]:
df_test_01_2022["RatecodeID"].value_counts()

1.0     2245776
2.0       65620
5.0       13234
99.0       8712
3.0        3934
4.0        3075
6.0          16
Name: RatecodeID, dtype: int64

In [22]:
#Based on the data dictionary, the possible values for RatecodeID should only be 1,2,3,4,5, and 6
#Remove the value of 99.0
RatecodeID = list(range(1,7))
df_test_01_2022 = df_test_01_2022[df_test_01_2022["RatecodeID"].isin(RatecodeID)]
df_test_01_2022["RatecodeID"].value_counts()

1.0    2245776
2.0      65620
5.0      13234
3.0       3934
4.0       3075
6.0         16
Name: RatecodeID, dtype: int64

In [23]:
df_test_01_2022["store_and_fwd_flag"].value_counts()

N    2280857
Y      50798
Name: store_and_fwd_flag, dtype: int64

In [24]:
df_test_01_2022["store_and_fwd_flag"].isna().sum()

0

In [25]:
df_test_01_2022["payment_type"].value_counts()

1    1826424
2     484440
4      10422
3      10368
5          1
Name: payment_type, dtype: int64

In [26]:
df_test_01_2022["payment_type"].isna().sum()

0

In [27]:
df_test_01_2022["fare_amount"].isna().sum()

0

In [28]:
df_test_01_2022["extra"].value_counts()

0.00     973573
0.50     392259
2.50     344411
1.00     316634
3.00     131909
          ...  
33.50         1
6.75          1
13.75         1
2.95          1
0.30          1
Name: extra, Length: 61, dtype: int64

In [29]:
df_test_01_2022["extra"].isna().sum()

0

In [30]:
df_test_01_2022["mta_tax"].value_counts()

 0.50     2303553
 0.00       15525
-0.50       12559
 0.07           7
-0.07           5
 0.35           1
 16.59          1
 0.08           1
 0.10           1
 0.32           1
 3.30           1
Name: mta_tax, dtype: int64

In [31]:
#remove all tax datapoint except 0.50
df_test_01_2022 = df_test_01_2022[df_test_01_2022["mta_tax"] == 0.50]
df_test_01_2022["mta_tax"].value_counts()

0.5    2303553
Name: mta_tax, dtype: int64

In [32]:
df_test_01_2022["improvement_surcharge"].value_counts()

0.3    2303421
0.0        132
Name: improvement_surcharge, dtype: int64

In [33]:
#remove all tax datapoint except 0.3
df_test_01_2022 = df_test_01_2022[df_test_01_2022["improvement_surcharge"] == 0.3]
df_test_01_2022["improvement_surcharge"].value_counts()

0.3    2303421
Name: improvement_surcharge, dtype: int64

In [34]:
df_test_01_2022["improvement_surcharge"].isna().sum()

0

In [35]:
df_test_01_2022["tip_amount"].value_counts()

0.00     550130
1.00     128379
2.00     105820
2.06      39112
1.96      38611
          ...  
29.60         1
28.74         1
17.13         1
19.34         1
24.28         1
Name: tip_amount, Length: 2488, dtype: int64

In [36]:
df_test_01_2022["tip_amount"].isna().sum()

0

In [37]:
df_test_01_2022["tolls_amount"].value_counts()

0.00     2184189
6.55      115410
3.00        1022
13.10        465
6.12         301
          ...   
9.30           1
14.25          1
3.70           1
73.08          1
8.08           1
Name: tolls_amount, Length: 215, dtype: int64

In [38]:
df_test_01_2022["tolls_amount"].isna().sum()

0

In [39]:
df_test_01_2022["total_amount"].isna().sum()

0

In [40]:
df_test_01_2022["congestion_surcharge"].value_counts()

2.50    2142818
0.00     160598
0.75          5
Name: congestion_surcharge, dtype: int64

In [41]:
df_test_01_2022["congestion_surcharge"].isna().sum()

0

In [42]:
#no need to remove the 0 airport fee
df_test_01_2022["airport_fee"].value_counts()

0.00    2149418
1.25     154003
Name: airport_fee, dtype: int64

In [43]:
df_test_01_2022["airport_fee"].isna().sum()

0

##### 4. Normalizing column names

In [44]:
df_test_01_2022 = clean_columns(df_test_01_2022, case="snake")
df_test_01_2022.head()

4 column names have been cleaned

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


##### 5. Normalizing and using appropriate column types for the respective data;

In [45]:
df_test_01_2022.dtypes

vendor_id                         int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
ratecode_id                     float64
store_and_fwd_flag               object
pu_location_id                    int64
do_location_id                    int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [46]:
type(df_test_01_2022.store_and_fwd_flag[0])

str

In [47]:
df_test_01_2022["store_and_fwd_flag"].replace(["Y", "N"],
                                             [1,0], inplace= True)

In [48]:
df_test_01_2022.head()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,0,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,0,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,0,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,0,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,0,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [49]:
df_test_01_2022.dtypes

vendor_id                         int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
ratecode_id                     float64
store_and_fwd_flag                int64
pu_location_id                    int64
do_location_id                    int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [50]:
df_test_01_2022 = df_test_01_2022.astype({"passenger_count":"int", 
                                          "ratecode_id":"int"})
df_test_01_2022.dtypes

vendor_id                         int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
ratecode_id                       int64
store_and_fwd_flag                int64
pu_location_id                    int64
do_location_id                    int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

##### 6. For Uber and Yellow Taxi data, removing trips that start and/or end outside of the following latitude/longitude coordinate box: (40.560445, -74.242330) and (40.908524, -73.717047).

In [51]:
#dependent to the first part of cleaning and filtering section

### Sampling

In [52]:
len(df_test_01_2022)

2303421

In [53]:
# Get a random sampling with 0.5 as a fraction
df_test_01_2022 = df_test_01_2022.sample(frac=0.5, random_state=1)

In [54]:
len(df_test_01_2022)*2

2303420

In [55]:
df_test_01_2022.head()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1813513,2,2022-01-24 16:32:52,2022-01-24 16:44:14,1,1.76,1,0,237,137,1,9.5,1.0,0.5,2.76,0.0,0.3,16.56,2.5,0.0
1999690,2,2022-01-26 18:37:00,2022-01-26 18:39:36,1,0.68,1,0,90,68,1,4.0,1.0,0.5,1.00,0.0,0.3,9.30,2.5,0.0
1956035,2,2022-01-26 11:51:36,2022-01-26 12:03:58,1,1.53,1,0,229,140,1,9.5,0.0,0.5,2.56,0.0,0.3,15.36,2.5,0.0
1290690,2,2022-01-18 15:14:47,2022-01-18 15:21:43,2,1.51,1,0,144,107,1,7.0,0.0,0.5,2.06,0.0,0.3,12.36,2.5,0.0
1270835,2,2022-01-18 11:47:27,2022-01-18 11:59:21,1,1.35,1,0,48,233,2,9.0,0.0,0.5,0.00,0.0,0.3,12.30,2.5,0.0


In [56]:
df_test_01_2022.vendor_id.value_counts()

2    821431
1    330279
Name: vendor_id, dtype: int64

In [60]:
data_gpd = gpd.read_file("taxi_zones.shp")
data_gpd

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


### Missing data

In [63]:
#will be tackled later on